In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import norm

In [5]:
# Cargamos el archivo ya limpio

df= pd.read_csv('marketing_customer_analysis(2).csv')

Already done in the round 2.

### Explore the data

In [6]:
df.shape
df_2= df.copy()

In [7]:
df_2.columns

Index(['unnamed', 'customer', 'state', 'customer lifetime value', 'response',
       'coverage', 'education', 'effective to date', 'employmentstatus',
       'gender', 'income', 'location code', 'marital status',
       'monthly premium auto', 'months since last claim',
       'months since policy inception', 'number of open complaints',
       'number of policies', 'policy type', 'policy', 'renew offer type',
       'sales channel', 'total claim amount', 'vehicle class', 'vehicle size',
       'vehicle type', 'Month'],
      dtype='object')

In [8]:
# Renombrar la columna 'unnamed: 0' a 'unnamed'
df_2.rename(columns={'unnamed: 0': 'unnamed'}, inplace=True)

In [9]:
df_2.info

<bound method DataFrame.info of        unnamed customer       state  customer lifetime value response  \
0            0  DK49336     Arizona              4809.216960       No   
1            1  KX64629  California              2228.525238       No   
2            2  LZ68649  Washington             14947.917300       No   
3            3  XL78013      Oregon             22332.439460      Yes   
4            4  QA50777      Oregon              9025.067525       No   
...        ...      ...         ...                      ...      ...   
10905    10905  FE99816      Nevada             15563.369440       No   
10906    10906  KX53892      Oregon              5259.444853       No   
10907    10907  TL39050     Arizona             23893.304100       No   
10908    10908  WA60547  California             11971.977650       No   
10909    10909  IV32877      Unknow              6857.519928   Unknow   

       coverage education effective to date employmentstatus gender  ...  \
0         Basic

In [10]:
# Visualizamos la columnas que son numericas
df_2._get_numeric_data()

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2


In [ ]:
df_2.describe

In [12]:
# Seleccionar todas las columnas categóricas
df_2_object= df_2.select_dtypes(include=['object', 'category'])
df_2_numerical= df_2.select_dtypes(np.number)

In [13]:
df_2_numerical

,unnamed,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,Month
0,0,4809.216960,48029,61,7.000000,52,0.000000,9,292.800000,2
1,1,2228.525238,0,64,3.000000,26,0.000000,1,744.924331,1
2,2,14947.917300,22139,100,34.000000,31,0.000000,2,480.000000,2
3,3,22332.439460,49078,97,10.000000,3,0.000000,2,484.013411,1
4,4,9025.067525,23675,117,15.149071,31,0.384256,7,707.925645,1
...,...,...,...,...,...,...,...,...,...,...
10905,10905,15563.369440,0,253,15.149071,40,0.384256,7,1214.400000,1
10906,10906,5259.444853,61146,65,7.000000,68,0.000000,6,273.018929,1
10907,10907,23893.304100,39837,201,11.000000,63,0.000000,2,381.306996,2
10908,10908,11971.977650,64195,158,0.000000,27,4.000000,6,618.288849,2
